In [0]:
import numpy as np
import math
import io
import pandas as pd
import seaborn as snb
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import metrics
from scipy.io import loadmat

In [0]:
data=pd.read_csv('ratings.csv')

In [0]:
total_item=36
total_user=36
D = 6
print(total_user)
print(total_item)

36
36


In [0]:
f=data.loc[data['movieId']<=total_item]
f=f.loc[data['userId']<=total_user]
#print(f)

In [0]:
from sklearn.utils import shuffle
f = shuffle(f)
#print(f)

In [0]:
k=10
# user matrix U
U=np.random.rand(k,total_user)

# item matrix V
V=np.random.rand(k,total_item)
print(U.shape)
print(V.shape)

# Rating matrix R
R=np.zeros((total_user,total_item),dtype=float)

(10, 36)
(10, 36)


In [0]:
# Block list to stop accessing current user block
userBlockList = [False]*(D+1)   # false -> not block    ,,   true --> block

# Block list to stop accessing current item block
itemBlockList = [False]*(D+1)

# waiting list for block p,q
global waitingList
waitingList = []

![picture](https://drive.google.com/uc?id=129VL6UxDNxdFuIbd3w890qaTab1auydF)

In [0]:
def matrixStore(UMat=None,VMat=None,p=-1,q=-1,UFetch=False,VFetch=False,UStore=False,VStore=False):
    # if worker want to store updated user block then this condition
    if UStore == True:
        U[:][p*(D-1):p*D] = UMat
        return

    # if worker want to store updated item block then this condition
    if VStore == True:
        V[:][q*(D-1):q*D] = VMat
        return
    
    # same for getch 
    if UFetch == True:
        while userBlockList[p] == True:
            val = 0
        return U[:][p*(D-1):p*D]
    
    if VFetch == True:  
        while itemBlockList[q] == True:
            val = 0
        return V[:][q*(D-1):q*D]

![picture](https://drive.google.com/uc?id=1WWi3cFm0owxFv_sr5DegVRyDUokEI8uC)

In [0]:
def worker(i=-1,j=-1,Rij=-1,p=-1,q=-1):
    
    # if i,j,Rij is received
    if(i != -1):
        j -= 1
        i -= 1
        R[i][j] = Rij
        return
    
    # if block p q is received then worker have predict new ratings
    if(p != -1):
        # fetching matrix
        up = matrixStore(UFetch=True,p=p,q=q)
        vq = matrixStore(VFetch=True,p=p,q=q)
        
        # predicting algo
        n=total_user
        m=total_item
        alpha=0.01
        lamda=10000

        for i in range(p*(D-1),p*D,1):
            for j in range(q*(D-1),q*D,1):
                if R[i][j]!=0:
                    ui=U[:,i]
                    vj=V[:,j]
                    #print(ui.shape,vj.shape)
                    ui=ui-(2*alpha)*(((np.matmul(ui.T,vj)-R[i][j])*vj)+(lamda/m)*ui)
                    vj=vj-(2*alpha)*(((np.matmul(ui.T,vj)-R[i][j])*ui)+(lamda/n)*vj)
                    U[:,i]=ui 
                    V[:,j]=vj
        
        # storing back updated block
        matrixStore(UStore=True,UMat=up)
        matrixStore(VStore=True,VMat=vq)

![picture](https://drive.google.com/uc?id=1lMQTS9jwICLPMSNqoY9djpPHb1NeMgkj)

In [0]:
## This is a master 
## fetching one by one data row
for index, row in f.iterrows():
    i = int(row['userId'])
    j = int(row['movieId'])
    Rij = row['rating']
    p = i//D 
    q = j//D
    #print(i,j)
    #print(p,q)
    
    # adding into waiting list
    waitingList.append((p,q))
    waitingList = list(set(waitingList))
    
    worker(i=i,j=j,Rij=Rij)
    
    if len(waitingList)>0:
        p,q = map(int,waitingList[0])
        waitingList.pop(0)
        
        #print(p,q)
        userBlockList[p] = False  # True in case of threading 
        itemBlockList[q] = False  # True in case of threading
        
        worker(p=p,q=q)
                
        userBlockList[p] = False
        itemBlockList[q] = False

/home/ajaybechara1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in multiply
/home/ajaybechara1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in subtract


In [0]:
print(R)

[[0. 0. 0. ... 0. 0. 0.]
 [5. 0. 2. ... 0. 0. 5.]
 [0. 0. 0. ... 5. 0. 4.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
